In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
SEED=1

In [3]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [4]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [5]:
##########################
ARCH = 'resnet18' # set the architecture to RESNET 18
########################
EPOCHS = 100
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=2
TRAINDIR="data/train"
VALDIR="data/val"

### Check if cuda is available here

In [6]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!
print(torch.cuda.get_device_name(0))

if torch.cuda.is_available():
    print(f'Device detected!')

Tesla T4
Device detected!


### Assign your GPU below

In [7]:
# Assign your GPU in this cell
# GPU = torch.cuda.device('cuda:0')
GPU = 0

In [8]:
# set your active device to your GPU in this cell
torch.cuda.set_device(GPU)

In [9]:
# enable algorithm optimization
cudnn.benchmark = True

### Fill in the heart of the train section below

In [10]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        # send the target to cuda device
        if GPU is not None:
          images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
          target = target.cuda(GPU, non_blocking=True)

        # compute output
        # output = model ?? images
        output = model(images)

        # compute loss 
        # loss = criterion, output, target
        loss = criterion(output, target)


        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        
        #### zero out gradients in the optimier
        ## optimizer ..??
        optimizer.zero_grad()
        
        ## backprop!
        ### loss... ???
        loss.backward()
        
        # update the weights!
        ## optimier .. ??
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [11]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    # model ???
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            
            ### send the images and target to cuda
            if GPU is not None:
              images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
              target = target.cuda(GPU, non_blocking=True)

            # compute output
            # output = model ??? images?
            output = model(images)

            # compute loss
            # loss  = criterion ?? output ?? target
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [12]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    # state ??? 
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [14]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [15]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [16]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [17]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [18]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [19]:
IMG_SIZE = 224
NUM_CLASSES = 1000

### Initialize the model using the architecture you selected above

In [20]:
# select the model
# model = ... 
model = models.__dict__[ARCH]()
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

### Send the model to the cuda device

In [21]:
# send the model to the cuda device.. 
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Instantiate the loss to cross entropy

In [22]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss().cuda(GPU)

### Instantiate the optimizer to SGD

In [23]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), LR,
                               momentum=MOMENTUM,
                               weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [24]:
# use CosineAnnealingLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [25]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

### Create the train dataset object

In [26]:
# use torchvision.datasets.CIFAR10
train_dataset = torchvision.datasets.ImageFolder(
    root=TRAINDIR, transform=transform_train
)

### Create the val dataset object

In [27]:
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [28]:
# use torchvision.datasets.CIFAR10
val_dataset = torchvision.datasets.ImageFolder(
    root=VALDIR, transform=transform_val
)

### Create the train dataloader

In [29]:
# fill this in
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=TRAIN_BATCH, 
                                           shuffle=True,
                                           num_workers=WORKERS, 
                                           pin_memory=True, 
                                           sampler=None)

### Create the c

In [30]:
# fill this in..
val_loader = torch.utils.data.DataLoader(val_dataset, 
                                         batch_size=VAL_BATCH, 
                                         shuffle=False,
                                         num_workers=WORKERS, 
                                         pin_memory=True, 
                                         sampler=None) 

In [31]:
best_acc1 = 0

In [32]:
for epoch in range(START_EPOCH, EPOCHS):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][    0/10010]	Time  8.394 ( 8.394)	Data  1.103 ( 1.103)	Loss 7.0408e+00 (7.0408e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)


Epoch: [0][   50/10010]	Time  0.375 ( 0.639)	Data  0.000 ( 0.022)	Loss 6.9409e+00 (7.1220e+00)	Acc@1   0.00 (  0.12)	Acc@5   0.00 (  0.64)


Epoch: [0][  100/10010]	Time  0.386 ( 0.577)	Data  0.000 ( 0.015)	Loss 6.8207e+00 (7.0071e+00)	Acc@1   0.78 (  0.20)	Acc@5   1.56 (  0.91)


Epoch: [0][  150/10010]	Time  0.599 ( 0.554)	Data  0.000 ( 0.013)	Loss 6.8112e+00 (6.9511e+00)	Acc@1   0.00 (  0.27)	Acc@5   1.56 (  1.10)


Epoch: [0][  200/10010]	Time  0.501 ( 0.541)	Data  0.000 ( 0.011)	Loss 6.8003e+00 (6.9108e+00)	Acc@1   0.78 (  0.32)	Acc@5   0.78 (  1.27)


...

Epoch: [36][  500/10010]	Time  0.726 ( 0.740)	Data  0.000 ( 0.012)	Loss 1.9111e+00 (1.8509e+00)	Acc@1  53.91 ( 57.66)	Acc@5  76.56 ( 79.98)


Epoch: [36][  550/10010]	Time  0.727 ( 0.739)	Data  0.000 ( 0.012)	Loss 1.6215e+00 (1.8513e+00)	Acc@1  58.59 ( 57.62)	Acc@5  87.50 ( 79.98)


Epoch: [36][  600/10010]	Time  0.728 ( 0.738)	Data  0.000 ( 0.012)	Loss 1.9780e+00 (1.8506e+00)	Acc@1  53.12 ( 57.57)	Acc@5  79.69 ( 79.97)


Epoch: [36][  650/10010]	Time  0.728 ( 0.737)	Data  0.000 ( 0.012)	Loss 1.7999e+00 (1.8503e+00)	Acc@1  64.06 ( 57.60)	Acc@5  82.81 ( 79.99)


Epoch: [36][  700/10010]	Time  0.734 ( 0.736)	Data  0.000 ( 0.011)	Loss 1.6443e+00 (1.8524e+00)	Acc@1  64.06 ( 57.59)	Acc@5  80.47 ( 79.99)
